In [1]:
# read test data from data/{dataname}/Queried_{dataname}_all_models_clean_test

import pandas as pd
import numpy as np
import os
import sys
import json
import re
import tiktoken

sys.path.append('src')

from FrugalGPT import optimizer
import FrugalGPT

/Users/sharkiefff/anaconda3/envs/dis-llm/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
supported_LLM = FrugalGPT.getservicename()
print("supported LLMs:",supported_LLM)
supported_LLM_names = [llm.split("/")[1] for llm in supported_LLM]
print("supported_LLM_names:", supported_LLM_names)

supported LLMs: ['google/gemini-1.5-flash-002', 'google/gemini-1.5-pro-002', 'google/gemini-1.0-pro', 'openaichat/gpt-4o-mini', 'openaichat/gpt-4o', 'azure/Phi-3-mini-4k-instruct', 'azure/Phi-3.5-mini-instruct', 'azure/Phi-3-small-8k-instruct', 'azure/Phi-3-medium-4k-instruct', 'deepinfra/llama-3-8B', 'deepinfra/llama-3-70B', 'deepinfra/mixtral-8x7B']
supported_LLM_names: ['gemini-1.5-flash-002', 'gemini-1.5-pro-002', 'gemini-1.0-pro', 'gpt-4o-mini', 'gpt-4o', 'Phi-3-mini-4k-instruct', 'Phi-3.5-mini-instruct', 'Phi-3-small-8k-instruct', 'Phi-3-medium-4k-instruct', 'llama-3-8B', 'llama-3-70B', 'mixtral-8x7B']


In [3]:
service_names = ['openaichat/gpt-4o-mini',
                'openaichat/gpt-4o',
                'google/gemini-1.5-flash-002',
                'google/gemini-1.5-pro-002',
                'google/gemini-1.0-pro',
                'azure/Phi-3-mini-4k-instruct',
                'azure/Phi-3.5-mini-instruct',
                'azure/Phi-3-small-8k-instruct',
                'azure/Phi-3-medium-4k-instruct',
                'deepinfra/llama-3-8B',
                'deepinfra/llama-3-70B',
                'deepinfra/mixtral-8x7B',
                ]

In [4]:
dataname = "OVERRULING"
# dataname = "AGNEWS"
# dataname = "HEADLINES"
# read data
test_data_df = pd.read_csv(f"data/{dataname}/Queried_{dataname}_all_models_clean_train.csv", header=0) # test

test_data_df.head()

,query_raw,query,ref_answer,gpt-4o-mini,gpt-4o,llama-3-8B,llama-3-70B,mixtral-8x7B,gemini-1.5-flash-002,gemini-1.0-pro,gemini-1.5-pro-002,Phi-3.5-mini-instruct,Phi-3-small-8k-instruct,Phi-3-mini-4k-instruct,Phi-3-medium-4k-instruct
0,Context: to the extent that these cases are in...,Please determine whether a sentence is overrul...,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes
1,Context: we therefore reverse the order denyin...,Please determine whether a sentence is overrul...,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes
2,"Context: see brown v. state,\nQuestion: Is it ...",Please determine whether a sentence is overrul...,no,no,no,no,no,no,no,no,no,no,no,no,no
3,"Context: at the very least, this court ought t...",Please determine whether a sentence is overrul...,no,no,no,no,no,no,no,no,no,yes,no,no,no
4,Context: the federal immigration judge and the...,Please determine whether a sentence is overrul...,yes,no,no,yes,no,no,yes,no,no,yes,no,yes,yes


In [5]:
test_data = []
for index, row in test_data_df.iterrows():
    query = row['query']
    ref_answer = row['ref_answer']
    _id = index
    model_answer = {}
    for model_name in supported_LLM_names:
        model_answer[model_name] = row[model_name]
    test_data.append([query, ref_answer, _id, model_answer])

In [6]:
print(test_data_df.iloc[1]["query"])
# calculate the number of words in the query
encoding = tiktoken.get_encoding('cl100k_base')
in_token_num = len(encoding.encode(test_data_df.iloc[303]["query"]))
print(in_token_num)

Please determine whether a sentence is overruling a prior decision (Yes or No) in the following statements.

Context: because jones/walker relates only to sufficiency of the evidence, we hereby disavow the language holding otherwise in sandoval.
Question: Is it overruling?
Answer: Yes

Context: according to napa auto parts, the straws drove the vehicle """"for approximately six [] weeks and [] for between 500 to 600 miles prior to the accident with no incidents.""""
Question: Is it overruling?
Answer: No

Context: we therefore reverse the order denying the motion to suppress and recede from any language in moskowitz which could be interpreted contrary to our holding in this case.
Question: Is it overruling?
Answer:
154


In [7]:
# calculate the average token length in query column using tiktoken
def get_avg_token_length(dataframe):
    token_num = 0
    for q in dataframe["query"]:
        token_num += len(tiktoken.get_encoding('cl100k_base').encode(q))
    return token_num / len(dataframe)
print(get_avg_token_length(test_data_df))

169.2025462962963


In [8]:
# read from config/serviceinfo_thrift.json, get the price per token for each model

with open("config/serviceinfo_thrift.json", "r") as f:
    serviceinfo = json.load(f)

def get_input_price_per_token(model_name):
    for service in serviceinfo:
        if model_name in serviceinfo[service]:
            return serviceinfo[service][model_name]["cost_input"]
    return None

def get_output_price_per_token(model_name):
    for service in serviceinfo:
        if model_name in serviceinfo[service]:
            return serviceinfo[service][model_name]["cost_output"]
    return None

def get_fixed_price(model_name):
    for service in serviceinfo:
        if model_name in serviceinfo[service]:
            return serviceinfo[service][model_name]["cost_fixed"]
    return None

print(get_input_price_per_token("gemini-1.5-flash-002"))
print(get_output_price_per_token("gpt-4o"))
print(get_fixed_price("gpt-4o"))

7.5e-08
1.5e-05
0


In [9]:
data = test_data
llm_vanilla = FrugalGPT.llmvanilla.LLMVanilla()

# 计算average cost in the test data for each model
for name in service_names:
    service_name = name
    sum_cost = 0
    for i in range(len(data)):
        query = data[i][0]
        cost = llm_vanilla.compute_cost(input_text=query, output_text="no", service_name=service_name)
        sum_cost += cost
        # print("data index is: ", data[i][2], "and cost for", service_name, " is: ", cost)
    print("average cost for", service_name, " is: ", sum_cost/len(data))

average cost for openaichat/gpt-4o-mini  is:  2.614583333333338e-05
average cost for openaichat/gpt-4o  is:  0.0008665277777777757
average cost for google/gemini-1.5-flash-002  is:  1.307291666666669e-05
average cost for google/gemini-1.5-pro-002  is:  0.0006065694444444452
average cost for google/gemini-1.0-pro  is:  8.665277777777774e-05
average cost for azure/Phi-3-mini-4k-instruct  is:  2.2659722222222156e-05
average cost for azure/Phi-3.5-mini-instruct  is:  2.2659722222222156e-05
average cost for azure/Phi-3-small-8k-instruct  is:  2.614583333333338e-05
average cost for azure/Phi-3-medium-4k-instruct  is:  2.9631944444444425e-05
average cost for deepinfra/llama-3-8B  is:  9.421805555555558e-06
average cost for deepinfra/llama-3-70B  is:  6.0006944444444445e-05
average cost for deepinfra/mixtral-8x7B  is:  4.1113333333333386e-05


In [13]:
# calculate the total cost of a cascaded models (depth = 3), by summing the cost of input and output tokens
# first read from /strategy/{dataname}_1015/cascade_strategy.json, to get the model_list with corresponding budget
# this file is like:
# {
    # "budget": {
    #     "1e-05": {
    #         "thres_list": [
    #             0.05691420375880523,
    #             0.10379833045938076,
    #             1.0
    #         ],
    #         "model_list": [
    #             "deepinfra/llama-3-8B",
    #             "azure/Phi-3-mini-4k-instruct",
    #             "openaichat/gpt-4o-mini"
    #         ],
    #         "quantile": [
    #             0.02565051282051282,
    #             0.02565051282051282
    #         ]
    #     },
    #     "5e-05": {

    # need to calculate the total cost of a cascaded models (depth = 3), by summing the cost of input and output tokens for each model
    # and use the quantile to decide the weight, e.g., 0.02565051282051282 for the first model, 0.02565051282051282 for using both the first and second model, 1-0.02565051282051282-0.02565051282051282 for using all three models
    

with open(f"strategy/{dataname}_1015/cascade_strategy.json", "r") as f:
    cascade_strategy = json.load(f)

def get_model_list(budget):
    # split and only need the model name, e.g., from 'openaichat/gpt-4o-mini' to 'gpt-4o-mini'
    model_list = cascade_strategy["budget"][budget]["model_list"]
    return [re.split("/", model)[1] for model in model_list]

def get_quantile(budget):
    # need to fill up the third value which is 1-quantile[0]-quantile[1]
    quantile = cascade_strategy["budget"][budget]["quantile"]
    quantile.append(1-quantile[0]-quantile[1])
    return quantile

print(get_model_list("1e-05"))
print(get_quantile("1e-05"))

['llama-3-8B', 'Phi-3-mini-4k-instruct', 'gpt-4o-mini']
[0.02565051282051282, 0.02565051282051282, 0.9486989743589744]


In [14]:
def get_total_cost(dataframe, budget):
    # need to calculate the total cost of a cascaded models (depth = 3), by summing the cost of input and output tokens for each model
    # and use the quantile to decide the weight, e.g., 0.02565051282051282 for the first model, 0.02565051282051282 for using both the first and second model, 1-0.02565051282051282-0.02565051282051282 for using all three models
    model_list = get_model_list(budget)
    use_first_cost = (get_input_price_per_token(model_list[0]) * get_avg_token_length(dataframe) + get_output_price_per_token(model_list[0]) * 1) * get_quantile(budget)[2]
    # use_first2_cost = (get_input_price_per_token(model_list[0]) * get_avg_token_length(dataframe) + get_output_price_per_token(model_list[0]) * 1 
    #                    + (get_input_price_per_token(model_list[1]) * get_avg_token_length(dataframe) + get_output_price_per_token(model_list[1]) * 1)) * get_quantile(budget)[1]
    # use_all3_cost = (get_input_price_per_token(model_list[0]) * get_avg_token_length(dataframe) + get_output_price_per_token(model_list[0]) * 1
                        # + (get_input_price_per_token(model_list[1]) * get_avg_token_length(dataframe) + get_output_price_per_token(model_list[1]) * 1)
                        # + (get_input_price_per_token(model_list[2]) * get_avg_token_length(dataframe) + get_output_price_per_token(model_list[2]) * 1)) * get_quantile(budget)[0]
    use_first2_cost = (get_input_price_per_token(model_list[1]) * get_avg_token_length(dataframe) + get_output_price_per_token(model_list[1]) * 1) * get_quantile(budget)[1]
    use_all3_cost = (get_input_price_per_token(model_list[2]) * get_avg_token_length(dataframe) + get_output_price_per_token(model_list[2]) * 1) * get_quantile(budget)[0]
    
    total_cost = use_first_cost + use_first2_cost + use_all3_cost
    return total_cost


def get_total_cost_cascaded(dataframe, budget):
    model_list = get_model_list(budget)
    total_cost = 0
    for i in range(len(model_list)):
        model = model_list[i]
        total_cost += get_input_price_per_token(model) * get_avg_token_length(dataframe)
        total_cost += get_output_price_per_token(model) * 1
        # if total_cost < float(budget):
        print(f"up to model {model} cost: {total_cost}")
            # continue
        # else:
            # break
    return total_cost

In [15]:
print(get_total_cost_cascaded(test_data_df, "1e-05"))
# print(get_total_cost(test_data_df, "0.001"))

up to model llama-3-8B cost: 9.361140046296297e-06
up to model Phi-3-mini-4k-instruct cost: 3.187747106481482e-05
up to model gpt-4o-mini cost: 5.785785300925927e-05
5.785785300925927e-05
